# AIDA Freediving Records

The Scope of that project is explore and display all freediving records from one of the world organization of freediving, AIDA in a series of D3.js visualizations. Such plateform could be extended to other datasets that share similar types of information (athlete name, country, record values (time, distance), location of the event, date).

1- Scraping the data from the website

2- Data preparation / cleaning / extension (separate name / country, get GPS locations, get gender...)

3- Early data exploration

4- D3.js data visualizations

# Scraping the data from www.aidainternational.org

Modules lxml and beautifulsoup parse the html in the 'page' variable returned from the website and store it in Beautiful Soup format. Import the library used to query a website and return the html to the variable 'page'. Convert list to dataframe with Pandas.

In [371]:
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import requests as rq

In [372]:
url = 'https://www.aidainternational.org/Ranking/Rankings?page=1'
page = rq.get(url)
soup = BeautifulSoup(page.content, "lxml") 

#Generate lists
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]
G=[]
H=[]
for row in soup.table.tbody.findChildren('tr'):
    cells = row.findAll('td')
    A.append(cells[0].find(text=True))
    B.append(cells[1].find(text=True))
    C.append(cells[2].find(text=True))
    D.append(cells[3].find(text=True))
    E.append(cells[4].find(text=True))
    F.append(cells[5].find(text=True))
    G.append(cells[6].find(text=True))
    H.append(cells[7].find(text=True))

#Convert list to data frame
df=pd.DataFrame(A,columns=['#'])
df['Athlete_Name']=B
df['Result']=C
df['Announced']=D
df['Points']=E
df['Penalties']=F
df['Date']=G
df['Place']=H


df

,#,Athlete_Name,Result,Announced,Points,Penalties,Date,Place
0,1.,Alexey MOLCHANOV,128,128,127,1,11/20/2012,Suunto Vertical Blue 2012
1,2.,Alexey MOLCHANOV,128,128,127,1,11/20/2012,Suunto Vertical Blue 2012
2,3.,Alexey MOLCHANOV,128,128,127,1,11/20/2012,Suunto Vertical Blue 2012
3,4.,Alexey MOLCHANOV,128,128,128,0,9/15/2013,AIDA INDIVIDUAL DEPHT WORLD CHAMPIONSHIP 2013
4,5.,Alexey MOLCHANOV,128,128,0,0,9/9/2013,2013 Kalamata Open
5,6.,Guillaume NÉRY,126,126,126,0,9/5/2015,2015 AIDA Depth World Championships Pre compet...
6,7.,Alexey MOLCHANOV,126,126,126,0,11/20/2012,Suunto Vertical Blue 2012
7,8.,Alexey MOLCHANOV,126,126,126,0,11/20/2012,Suunto Vertical Blue 2012
8,9.,Alexey MOLCHANOV,126,126,126,0,11/20/2012,Suunto Vertical Blue 2012
9,10.,Guillaume NÉRY,125,125,125,0,9/9/2013,2013 Kalamata Open


In [13]:
df.shape 

(20, 8)

In [3]:
#page = rq.get('https://www.aidainternational.org/Ranking/Rankings?page=1')

In [4]:
#page.status_code 

200

NB: a status code starting with a 2 generally indicates success, and a code starting with a 4 or a 5 indicates an error.

In [370]:
#page.content

In [7]:
#soup = BeautifulSoup(page.content, "lxml")
#soup

In [373]:
#print(soup.prettify())

The text data 'td' I am interested in are in the body 'tbody' of a table, two tags down with each row 'tr'

In [374]:
#soup.table.tbody.findChildren('tr')

Dataframe 'df' contains 20 first rows from page 1. Now, I will like to collect data from all 336 pages available for the discipline CWT - constant weight

-

# Webscraping multiple pages

In [14]:
#base_url = 'https://www.aidainternational.org/Ranking/Rankings?page='
#r = rq.get(base_url)

In [77]:
#for i in range(2):      # Number of pages plus one 
#    url = '{}{}'.format(base_url, i)
#    r = rq.get(url)
#    new_soup = BeautifulSoup(r.content, "lxml")    

In [277]:
#url_list = ['{}{}'.format(base_url, i) for i in range(1,5)]
#url_list

['https://www.aidainternational.org/Ranking/Rankings?page=1',
 'https://www.aidainternational.org/Ranking/Rankings?page=2',
 'https://www.aidainternational.org/Ranking/Rankings?page=3',
 'https://www.aidainternational.org/Ranking/Rankings?page=4']

Generate a list with all Urls e.g. max_pages=8

In [428]:
base_url = 'https://www.aidainternational.org/Ranking/Rankings?page='

def url_generator(max_pages):
    urls = ['{}{}'.format(base_url, i) for i in range(1, max_pages + 1)]
    return urls

In [429]:
urls = url_generator(6)
urls

['https://www.aidainternational.org/Ranking/Rankings?page=1',
 'https://www.aidainternational.org/Ranking/Rankings?page=2',
 'https://www.aidainternational.org/Ranking/Rankings?page=3',
 'https://www.aidainternational.org/Ranking/Rankings?page=4',
 'https://www.aidainternational.org/Ranking/Rankings?page=5',
 'https://www.aidainternational.org/Ranking/Rankings?page=6']

In [430]:
#def make_soup(urls):
    #for url_ in urls:
     #   print "Processing {}...".format(url_)
     #   page = rq.get(url_)
     #   new_soup = BeautifulSoup(page.content, "lxml")
    #return new_soup

In [446]:
def make_soup(max_pages):
    for i in range(1, max_pages + 1): # Number of pages plus one
        print "Processing {}...".format(i) 
        url = '{}{}'.format(base_url, i)
        page = rq.get(url)
        new_soup = BeautifulSoup(page.content)
        test = new_soup.table.tbody.findAll('td')
    return test

In [451]:
#make_soup(6)

In [450]:
def make_soup(max_pages):
    for i in range(1, max_pages + 1): # Number of pages plus one
        print "Processing {}...".format(i) 
        url = '{}{}'.format(base_url, i)
        page = rq.get(url)
        new_soup = BeautifulSoup(page.content)

        #Generate lists
        data_list = [A, B, C, D, E, F, G, H]
        for row in new_soup.table.tbody.findChildren('tr'):
            cells = row.findAll('td')

            A.append(cells[0].find(text=True))
            B.append(cells[1].find(text=True))
            C.append(cells[2].find(text=True))
            D.append(cells[3].find(text=True))
            E.append(cells[4].find(text=True))
            F.append(cells[5].find(text=True))
            G.append(cells[6].find(text=True))
            H.append(cells[7].find(text=True))

#Convert list to data frame
df=pd.DataFrame(A,columns=['#'])
df['Athlete_Name']=B
df['Result']=C
df['Announced']=D
df['Points']=E
df['Penalties']=F
df['Date']=G
df['Place']=H

df

,#,Athlete_Name,Result,Announced,Points,Penalties,Date,Place
0,101.,Ryuzo SHINOMIYA,105,105,105,0,11/20/2012,Suunto Vertical Blue 2012
1,102.,Guillaume NÉRY,105,105,104,1,6/6/2012,5th Mediterranean World Cup
2,103.,Guillaume NÉRY,105,105,104,1,6/6/2012,5th Mediterranean World Cup
3,104.,Guillaume NÉRY,105,105,105,0,6/20/2015,Nice Abyss Contest 2015
4,105.,Morgan BOURCHIS,105,105,105,0,9/15/2013,AIDA INDIVIDUAL DEPHT WORLD CHAMPIONSHIP 2013
5,106.,Alexey MOLCHANOV,105,105,105,0,11/1/2015,Big Blue
6,107.,William TRUBRIDGE,105,105,105,0,7/6/2010,7th AIDA Team World Championship 2010
7,108.,William TRUBRIDGE,105,119,90,15,4/17/2011,Vertical Blue 2011
8,109.,Homar LEUCI,105,105,0,0,4/22/2016,Vertical Blue 2016
9,110.,Homar LEUCI,105,105,105,0,4/22/2016,Vertical Blue 2016


In [452]:
def make_soup(max_pages):
    for i in range(1, max_pages + 1): # Number of pages plus one
        print "Processing {}...".format(i) 
        url = '{}{}'.format(base_url, i)
        page = rq.get(url)
        new_soup = BeautifulSoup(page.content)
        for row in new_soup.table.tbody.findChildren('tr'):
            print row.text

In [453]:
make_soup(3)

Processing 1...

1.
Alexey MOLCHANOV (Russian Federation)

128 m

128
127
1
11/20/2012
Suunto Vertical Blue 2012


2.
Alexey MOLCHANOV (Russian Federation)

128 m

128
127
1
11/20/2012
Suunto Vertical Blue 2012


3.
Alexey MOLCHANOV (Russian Federation)

128 m

128
127
1
11/20/2012
Suunto Vertical Blue 2012


4.
Alexey MOLCHANOV (Russian Federation)

128 m

128
128
0
9/15/2013
AIDA INDIVIDUAL DEPHT WORLD CHAMPIONSHIP 2013


5.
Alexey MOLCHANOV (Russian Federation)

128 m

128
0
0
9/9/2013
2013 Kalamata Open


6.
Guillaume NÉRY (France)

126 m

126
126
0
9/5/2015
2015 AIDA Depth World Championships Pre competition 


7.
Alexey MOLCHANOV (Russian Federation)

126 m

126
126
0
11/20/2012
Suunto Vertical Blue 2012


8.
Alexey MOLCHANOV (Russian Federation)

126 m

126
126
0
11/20/2012
Suunto Vertical Blue 2012


9.
Alexey MOLCHANOV (Russian Federation)

126 m

126
126
0
11/20/2012
Suunto Vertical Blue 2012


10.
Guillaume NÉRY (France)

125 m

125
125
0
9/9/2013
2013 Kalamata Open


11.
Da